In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/test_identity.csv
/kaggle/input/sample_submission.csv
/kaggle/input/train_identity.csv
/kaggle/input/train_transaction.csv
/kaggle/input/test_transaction.csv


### train_transaction.csv 파일과 train_identity.csv 파일 비교분석

두 파일의 공통점은 'TransactionID' 이다. 공통키 역할을 한다.

그런데 train_transaction.csv 파일의 TransactionID는 연속적인 반면에

train_identity.csv 파일의 TransactionID는 불연속적이다.

그래서 두 파일에 모두 존재하는 TransactionID를 기준으로 두 테이블을 합쳐야한다.

In [3]:
# 읽기모드로 train용 *.csv파일을 오픈한다.
train_transaction= pd.read_csv('../input/train_transaction.csv')
train_identity = pd.read_csv('../input/train_identity.csv')

# 합치기 전에 두 csv파일의 shape에 대해 알아보자.
print('train_transaction.shape: ',train_transaction.shape)
print('train_identity.shape: ',train_identity.shape)

train_transaction.shape:  (590540, 394)
train_identity.shape:  (144233, 41)


상대적으로 train_identity에 비해 

train_transaction이 더 큰 데이터를 보유하고 있음을 알 수있다.

합치기 전에 DataFrame.info() 메소드를 이용하여 

각 csv파일에 대한 정보를 탐색해본다.

In [15]:
# 읽기모드로 test용 *.csv파일을 오픈한다.
test_transaction= pd.read_csv('../input/test_transaction.csv')
test_identity= pd.read_csv('../input/test_identity.csv')

# 두 csv파일의 shape를 찍어본다.
print('test_transaction.shape: ', test_transaction.shape)
print('test_identity.shape: ', test_identity.shape)

test_transaction.shape:  (506691, 393)
test_identity.shape:  (141907, 41)


마찬가지로 test_transaction이 test_identity보다

더 많은 데이터셋을 보유하고 있다.

**test_transaction**: **506691**개의 행, 393개의 컬럼

**test_identity**   : **141907**개의 행, 41 개의 컬럼

In [8]:
# train_transaction
# train_transaction의 row의 개수 : 590540 개 / 컬럼개수: 394개
train_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [9]:
# train_identity
# train_identity의 row의 개수 : 144233 개 / 컬럼개수: 41개
train_identity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
TransactionID    144233 non-null int64
id_01            144233 non-null float64
id_02            140872 non-null float64
id_03            66324 non-null float64
id_04            66324 non-null float64
id_05            136865 non-null float64
id_06            136865 non-null float64
id_07            5155 non-null float64
id_08            5155 non-null float64
id_09            74926 non-null float64
id_10            74926 non-null float64
id_11            140978 non-null float64
id_12            144233 non-null object
id_13            127320 non-null float64
id_14            80044 non-null float64
id_15            140985 non-null object
id_16            129340 non-null object
id_17            139369 non-null float64
id_18            45113 non-null float64
id_19            139318 non-null float64
id_20            139261 non-null float64
id_21            5159 non-null float64
id_2

In [17]:
# test_transaction.info()
test_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506691 entries, 0 to 506690
Columns: 393 entries, TransactionID to V339
dtypes: float64(376), int64(3), object(14)
memory usage: 1.5+ GB


In [18]:
# test_identity.info()
test_identity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141907 entries, 0 to 141906
Data columns (total 41 columns):
TransactionID    141907 non-null int64
id_01            141907 non-null float64
id_02            136976 non-null float64
id_03            66481 non-null float64
id_04            66481 non-null float64
id_05            134750 non-null float64
id_06            134750 non-null float64
id_07            5059 non-null float64
id_08            5059 non-null float64
id_09            74338 non-null float64
id_10            74338 non-null float64
id_11            136778 non-null float64
id_12            141907 non-null object
id_13            130286 non-null float64
id_14            71357 non-null float64
id_15            136977 non-null object
id_16            125747 non-null object
id_17            135966 non-null float64
id_18            50875 non-null float64
id_19            135906 non-null float64
id_20            135633 non-null float64
id_21            5059 non-null float64
id_2

train_transaction 컬럼개수: 394개

test_transaction 컬럼개수: 393개

1개의 차이는 결과레이블인 **isFraud** 컬럼이다.

train_identity 컬럼개수: 41개

test_identity 컬럼개수 : 41개

In [12]:
# train_transaction과 train_identity의 공통키인 
# TransactionID의 개수를 카운트한다.
print('train_transaction의 TransactionID 개수:', \
     train_transaction['TransactionID'].count())

print('train_identity의 TransactionID 개수: ', \
     train_identity['TransactionID'].count())

train_transaction의 TransactionID 개수: 590540
train_identity의 TransactionID 개수:  144233


TransactionID 의 개수만을 세보면서 알 수 있는 것은 

각각의 csv파일의 TransactionID의 개수는 

각 csv 파일의 행(ROW)의 개수와 동일하다는 것이다.

그렇다면 train_transaction에서 

train_identity와 겹치는 TransactionID가 몇개인지 확인해보자.

동일한 방법으로 test_transaction 과test_identity와 겹치는 

TransactionID가 몇 개인지도 확인해보자.



In [16]:
print('train_transaction, train_identity 중 겹치는 TransactionID 개수\n: ',\
      np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'])))

print('test_transaction, test_identity 중 겹치는 TransactionID 개수\n: ',\
      np.sum(test_transaction['TransactionID'].isin(test_identity['TransactionID'])))


train_transaction, train_identity 중 겹치는 TransactionID 개수
:  144233
test_transaction, test_identity 중 겹치는 TransactionID 개수
:  141907


#### 공통키 TransactionID를 기준으로 train(test)_transaction 와 train(test)_identity 를 합쳐본다.

In [24]:
# train_identity의 TransactionID를 기준으로 데이터셋을 합쳐보자.
train_merged = pd.merge(train_transaction, train_identity, 
                        on='TransactionID', how='right')

test_merged= pd.merge(test_transaction, test_identity,
                     on='TransactionID', how='right' )

In [25]:
# train_identity를 기준으로 merge시켜봤다. 데이터의 손실이 크겠지만.. 그래도 해보자...
train_merged.head(3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0,86506,50.000,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,0,86535,15.000,H,2803,100.0,150.0,visa,226.0,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,0,86549,75.887,C,16496,352.0,117.0,mastercard,134.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


# EDA